## Importing and testing

In [63]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [64]:
es.ping()

True

## Creating index

In [66]:
es_index = 'movies'

In [67]:
movie_data_mapping = {
  "mappings": {
    "properties": {
      "name": {
        "type": "completion"
      },
        
         "year": {
        "type": "integer"
      }
    }
  }         
}

In [68]:
es.indices.create(es_index,movie_data_mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'}

## Indexing Data

In [61]:
data = [{ "name": "Spider-Man: Far From Home", "year": "2019"},
{ "name": "Avengers: Endgame", "year": "2019"},
{ "name": "Captain Marvel", "year": "2019"},
{ "name": "Ant-man and the Wasp", "year": "2018"},
{ "name": "Avengers: Infinity War", "year": "2018"},
{ "name": "Black Panther", "year": "2018"},
{ "name": "Thor: Ragnarok", "year": "2017"},
{ "name": "Spider-Man: Homecoming", "year": "2017"},
{ "name": "Guardians of the Galaxy Vol 2", "year": "2017"},
{ "name": "Doctor Strange", "year": "2016"},
{ "name": "Captain America: Civil War", "year": "2016"},
{ "name": "Ant-Man", "year": "2015"},
{ "name": "Avengers: Age of Ultron", "year": "2015"},
{ "name": "Guardians of the Galaxy", "year": "2014"},
{ "name": "Captain America: The Winter Soldier", "year": "2014"},
{ "name": "Thor: The Dark World", "year": "2013"},
{ "name": "Iron Man 3", "year": "2013"},
{ "name": "Marvel's The Avengers", "year": "2012"},
{ "name": "Captain America: The First Avenger", "year": "2011"},
{ "name": "Thor", "year": "2011"},
{ "name": "Iron Man 2", "year": "2010"},
{ "name": "The Incredible Hulk", "year": "2008"},
{ "name": "Iron Man", "year": "2008"}]

In [69]:
[es.index(index=es_index,  body=x) for x in data]

[{'_index': 'movies',
  '_type': '_doc',
  '_id': 'uebMWoQBIpQmeSUKusuC',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 0,
  '_primary_term': 1},
 {'_index': 'movies',
  '_type': '_doc',
  '_id': 'uubMWoQBIpQmeSUKusuU',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 1,
  '_primary_term': 1},
 {'_index': 'movies',
  '_type': '_doc',
  '_id': 'u-bMWoQBIpQmeSUKusug',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 2,
  '_primary_term': 1},
 {'_index': 'movies',
  '_type': '_doc',
  '_id': 'vObMWoQBIpQmeSUKusur',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 3,
  '_primary_term': 1},
 {'_index': 'movies',
  '_type': '_doc',
  '_id': 'vebMWoQBIpQmeSUKusuz',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'fa

## Checking data

In [70]:
es.search(index=es_index, body={"query": {"match_all": {}}},size=2)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 23, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'movies',
    '_type': '_doc',
    '_id': 'uebMWoQBIpQmeSUKusuC',
    '_score': 1.0,
    '_source': {'name': 'Spider-Man: Far From Home', 'year': '2019'}},
   {'_index': 'movies',
    '_type': '_doc',
    '_id': 'uubMWoQBIpQmeSUKusuU',
    '_score': 1.0,
    '_source': {'name': 'Avengers: Endgame', 'year': '2019'}}]}}

## Searching

### Highlights

In [72]:
body = {
            "query": {
                "match": { 'name': {'query': "thor"}
                         }

            }
        }

In [73]:
sources = ["name"]

In [74]:
body["highlight"] = {
        "pre_tags": ["<em class='highlight'>"],
        "post_tags": ["</em>"],
        'fields': {x: {'number_of_fragments': 0} for x in sources}
    }

In [75]:
es.search(index=es_index, body=body)

{'took': 33,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 2.7725885,
  'hits': [{'_index': 'movies',
    '_type': '_doc',
    '_id': 'zObMWoQBIpQmeSUKu8s0',
    '_score': 2.7725885,
    '_source': {'name': 'Thor', 'year': '2011'},
    'highlight': {'name': ["<em class='highlight'>Thor</em>"]}}]}}

### Auto complete

In [76]:
body = {
          "suggest": {
            "movie-suggest": {
              "prefix": "thor",
              "completion": {
                "field": "name"
              }
            }
          }
        }

In [77]:
es.search(index=es_index, body=body)

{'took': 25,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'suggest': {'movie-suggest': [{'text': 'thor',
    'offset': 0,
    'length': 4,
    'options': [{'text': 'Thor',
      '_index': 'movies',
      '_type': '_doc',
      '_id': 'zObMWoQBIpQmeSUKu8s0',
      '_score': 1.0,
      '_source': {'name': 'Thor', 'year': '2011'}},
     {'text': 'Thor: Ragnarok',
      '_index': 'movies',
      '_type': '_doc',
      '_id': 'v-bMWoQBIpQmeSUKusvG',
      '_score': 1.0,
      '_source': {'name': 'Thor: Ragnarok', 'year': '2017'}},
     {'text': 'Thor: The Dark World',
      '_index': 'movies',
      '_type': '_doc',
      '_id': 'yObMWoQBIpQmeSUKu8sU',
      '_score': 1.0,
      '_source': {'name': 'Thor: The Dark World', 'year': '2013'}}]}]}}

In [78]:
body = {
  "suggest": {
    "movie-suggest": {
      "prefix": "iro",
      "completion": {
        "field": "name",
        "size": 10
      }
    }
  }
}

In [79]:
es.search(index=es_index, body=body)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'suggest': {'movie-suggest': [{'text': 'iro',
    'offset': 0,
    'length': 3,
    'options': [{'text': 'Iron Man',
      '_index': 'movies',
      '_type': '_doc',
      '_id': 'z-bMWoQBIpQmeSUKu8tM',
      '_score': 1.0,
      '_source': {'name': 'Iron Man', 'year': '2008'}},
     {'text': 'Iron Man 2',
      '_index': 'movies',
      '_type': '_doc',
      '_id': 'zebMWoQBIpQmeSUKu8s9',
      '_score': 1.0,
      '_source': {'name': 'Iron Man 2', 'year': '2010'}},
     {'text': 'Iron Man 3',
      '_index': 'movies',
      '_type': '_doc',
      '_id': 'yebMWoQBIpQmeSUKu8sc',
      '_score': 1.0,
      '_source': {'name': 'Iron Man 3', 'year': '2013'}}]}]}}